In [4]:
# Packages & libraries 
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt 
import seaborn as sns

%matplotlib inline

# To install sklearn type "pip install numpy scipy scikit-learn" to the anaconda terminal

# To change scientific numbers to float
np.set_printoptions(formatter={'float_kind':'{:f}'.format})

# Increases the size of sns plots
sns.set(rc={'figure.figsize':(8,6)})

#ARIMA
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt

from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn import linear_model

import pmdarima as pm
from pmdarima.model_selection import train_test_split

#from fbprophet import Prophet

from pandas import to_datetime

import itertools
import warnings

import datetime
from datetime import datetime
warnings.filterwarnings('ignore')

# installing auto arima
# pip install --user pmdarima

# installing fbprophet
# pip install --user fbprophet
# !pip install --upgrade plotly

In [2]:
pip install --user pmdarima

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install --user fbprophet

  Running setup.py clean for fbprophet
  Created wheel for pymeeus: filename=PyMeeus-0.5.11-py3-none-any.whl size=730973 sha256=06673c5df81f8a10646167b6b5204b0778a014c871f7fe11660cbd1d548b3c49
  Stored in directory: c:\users\prati\appdata\local\pip\cache\wheels\a0\8b\b2\810ae5a6f970c8be4725353400d643c90de1c0f023a9884ee7
Successfully built pymeeus
Failed to build fbprophet
    Running setup.py install for fbprophet: started
    Running setup.py install for fbprophet: finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\prati\anaconda3\python.exe' -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\prati\\AppData\\Local\\Temp\\pip-install-r2rfyflf\\fbprophet_25c895e990f9491d91c58cf718c9f6f1\\setup.py'"'"'; __file__='"'"'C:\\Users\\prati\\AppData\\Local\\Temp\\pip-install-r2rfyflf\\fbprophet_25c895e990f9491d91c58cf718c9f6f1\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\prati\AppData\Local\Temp\pip-wheel-9uck9425'
       cwd: C:\Users\prati\AppData\Local\Temp\pip-install-r2rfyflf\fbprophet_25c895e990f9491d91c58cf718c9f6f1\
  Complete output (44 lines):
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib
  creating build\lib\fbprophet
  creating build\lib\fbprophet\stan_model
  Traceback (most recent call last):
    File 

### Problem Formulation - What are we trying to solve?

We want to use machine learning to predict call volumes for 20 different call intents.

## Reading Raw Data

In [5]:
df= pd.read_csv('C:\\Users\\prati\\American Express AI CHALLENGE\\Data\\df_train_callvolume (1).csv')
df

,INTERVAL,Account,AccountClose,AccountOnline,Account_Issue,ApplicationStatus,BillingQuery,CardDeclined,Card_Compromise,ChargeReview,...,DataBreach,Entertainment,FraudQuery,Online_Login,PaymentExecute,Payments,Representative,Rewards,Statement,Travel
0,2018-01-01 00:00,5,27,8,2,3,17,15,0,37,...,0,0,21,13,97,22,165,19,1,19
1,2018-01-01 00:15,2,15,5,1,0,8,18,2,30,...,0,0,25,2,93,19,138,23,5,20
2,2018-01-01 00:30,2,20,7,0,1,4,16,1,28,...,0,0,13,9,87,22,166,14,2,27
3,2018-01-01 00:45,1,22,3,0,2,10,13,0,31,...,0,0,20,7,79,26,155,12,2,33
4,2018-01-01 01:00,1,18,4,0,1,6,17,2,26,...,0,0,25,4,94,14,136,16,2,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35035,2018-12-31 22:45,3,77,21,0,3,48,36,0,60,...,0,0,56,27,200,37,421,57,9,56
35036,2018-12-31 23:00,3,59,17,2,2,38,24,1,62,...,0,0,46,14,215,47,361,33,9,60
35037,2018-12-31 23:15,9,64,12,0,1,37,29,0,67,...,0,0,58,23,178,26,353,35,8,37
35038,2018-12-31 23:30,2,55,13,2,1,27,24,2,39,...,1,1,43,14,178,46,314,33,8,48


In [6]:
print(df.shape)

(35040, 21)


In [7]:
df.head()

,INTERVAL,Account,AccountClose,AccountOnline,Account_Issue,ApplicationStatus,BillingQuery,CardDeclined,Card_Compromise,ChargeReview,...,DataBreach,Entertainment,FraudQuery,Online_Login,PaymentExecute,Payments,Representative,Rewards,Statement,Travel
0,2018-01-01 00:00,5,27,8,2,3,17,15,0,37,...,0,0,21,13,97,22,165,19,1,19
1,2018-01-01 00:15,2,15,5,1,0,8,18,2,30,...,0,0,25,2,93,19,138,23,5,20
2,2018-01-01 00:30,2,20,7,0,1,4,16,1,28,...,0,0,13,9,87,22,166,14,2,27
3,2018-01-01 00:45,1,22,3,0,2,10,13,0,31,...,0,0,20,7,79,26,155,12,2,33
4,2018-01-01 01:00,1,18,4,0,1,6,17,2,26,...,0,0,25,4,94,14,136,16,2,25


## Data Pre-Processing Phase

In [8]:
df.isnull().sum()

INTERVAL             0
Account              0
AccountClose         0
AccountOnline        0
Account_Issue        0
ApplicationStatus    0
BillingQuery         0
CardDeclined         0
Card_Compromise      0
ChargeReview         0
Complaint            0
DataBreach           0
Entertainment        0
FraudQuery           0
Online_Login         0
PaymentExecute       0
Payments             0
Representative       0
Rewards              0
Statement            0
Travel               0
dtype: int64

In [10]:
df['Account'], df['Statement'].isnull().unique()

(0        5
 1        2
 2        2
 3        1
 4        1
         ..
 35035    3
 35036    3
 35037    9
 35038    2
 35039    8
 Name: Account, Length: 35040, dtype: int64,
 array([False]))

In [13]:
# Investigate all the elements whithin each Feature 

for column in df:
    unique_vals = np.unique(df[column])
    nr_values = len(unique_vals)
    if nr_values < 20:
        print('The number of values for feature {} :{} -- {}'.format(column, nr_values,unique_vals))
    else:
        print('The number of values for feature {} :{}'.format(column, nr_values))

The number of values for feature INTERVAL :35040
The number of values for feature Account :34
The number of values for feature AccountClose :162
The number of values for feature AccountOnline :84
The number of values for feature Account_Issue :14 -- [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13]
The number of values for feature ApplicationStatus :14 -- [ 0  1  2  3  4  5  6  7  8  9 10 11 12 14]
The number of values for feature BillingQuery :189
The number of values for feature CardDeclined :111
The number of values for feature Card_Compromise :29
The number of values for feature ChargeReview :152
The number of values for feature Complaint :10 -- [0 1 2 3 4 5 6 7 8 9]
The number of values for feature DataBreach :36
The number of values for feature Entertainment :79
The number of values for feature FraudQuery :148
The number of values for feature Online_Login :81
The number of values for feature PaymentExecute :471
The number of values for feature Payments :174
The number of values for fea

In [17]:
# Filtering out the data we need 
forecast_data = df[df['Account'] == 'Account, (INTERVAL)']

forecast_data = forecast_data[~forecast_data['INTERVAL'].isin(['Account', 'AccountClose', 'AccountOnline', 'Account_Issue', 'ApplicationStatus', 'BillingQuery'])]

forecast_data.head()

,INTERVAL,Account,AccountClose,AccountOnline,Account_Issue,ApplicationStatus,BillingQuery,CardDeclined,Card_Compromise,ChargeReview,...,DataBreach,Entertainment,FraudQuery,Online_Login,PaymentExecute,Payments,Representative,Rewards,Statement,Travel


In [18]:
# Limiting the columns we need
forecast_data = forecast_data[['Account','AccountClose','AccountOnline']]
forecast_data.head()

,Account,AccountClose,AccountOnline


In [24]:
# Changing the time to date - Most Time Series Models require date structure
#forecast_data['INTERVAL'] = forecast_data['INTERVAL'].astype(str)  + '-01-01'
#forecast_data['INTERVAL'] = pd.to_datetime(forecast_data['INTERVAL'])

#forecast_data.head()
df.index = pd.to_datetime(df.index)

In [5]:
X_train = forecast_data[forecast_data['Account'] < '2018-01-01']
X_valid = forecast_data[forecast_data['Account'] >= '2018-12-31']

print('X_train Shape', X_train.shape)
print('X_Valid Shape', X_valid.shape)

X_train['Account'].unique()

NameError: name 'forecast_data' is not defined

In [6]:
########################################################################################################
####################################### Running ARIMA #####################################
########################################################################################################

# filtering the train data
au = X_train[X_train['INTERVAL'] == 'Account']
au2 = au[['INTERVAL', 'value']]

#filtering the valid data
au_v = X_valid[X_valid['INTERVAL'] == 'Account']
au_v2 = au_v[['INTERVAL', 'value']]

# set the date into index
au2.set_index('INTERVAL', inplace=True)
au_v2.set_index('INTERVAL', inplace=True)

# Setting the Index for 20 intents
index_20_years = pd.date_range(au2.index[-1], freq='AS', periods = 7) 

index_20_years

NameError: name 'X_train' is not defined

In [7]:
# Running ARIMA with random numbers
model_arima = ARIMA(au2, order=(0,2,3)) # start 2-0-0 and move to the best
model_arima_fit = model_arima.fit(disp=-1)

# Saving ARIMA predictions
fcast1 = model_arima_fit.forecast(7)[0]

# Passing the same index as the others
fcast1 = pd.Series(fcast1, index=index_7_years)
fcast1 = fcast1.rename("Arima") 

# Ploting the predictions
fig, ax = plt.subplots(figsize=(15,5))
chart = sns.lineplot(x='INTERVAL', y='value', data = au)
chart.set_title('AU')
fcast1.plot(ax=ax, color='red', marker="o", legend=True)
au_v2.plot(ax=ax, color='blue', marker="o", legend=True)

# Printing the AI
print('The MSE of ARIMA is:', mean_squared_error(au_v['value'].values, fcast1.values, squared=False))

NameError: name 'ARIMA' is not defined